# 04. Creación de grafo de las estaciones del sistema metro de CDMX

Carga de librerías

In [42]:
import json
from datetime import datetime, timedelta

import networkx as nx
import pandas as pd

import plotly.graph_objects as go

Lectura de archivo json con los tiempos de traslado entre estaciones consecutivas

In [43]:
json_file = "../output_metro/travel_times_metro.json"
with open(json_file) as input_json:
    dict_times_metro = json.load(input_json)

Construimos un par de diccionarios:
- ``travel_times_metro``: contendra información sobre las estaciones, su única conexión consecutiva y el tiempo de traslado entre el par de estaciones.
- ``location_stations``: contendrá la localización geográfica de cada estación

In [44]:
travel_times_metro = dict()
location_stations = dict()

for route_id in dict_times_metro.keys():
    stations_data = dict_times_metro[route_id]
    converted_route_times = list(map(lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second), [datetime.strptime(stat_time[1][4:], "%HH%MM%SS") for stat_time in stations_data]))
    
    travel_times_metro[route_id] = [(stations_data[n][0], converted_diff_time) for n, converted_diff_time in enumerate(converted_route_times)]

    for station_data in stations_data:
        location_stations[station_data[0]] = tuple(station_data[2:])

Inciamos la construcción y definición del grafo no dirigido de todas las estaciones del sistema de metro de la CDMX

In [45]:
# Inicializamos el diccionario
metro_network = dict()

# Iniciamos la insertacion de nodos por cada ruta
route_ids = travel_times_metro.keys()
for route_id in route_ids:

    # Capturamos las estaciones por ruta y sus tiempos de traslado a su estacion consecutiva
    stations, travel_times = zip(*travel_times_metro[route_id])
    NumStations = len(stations)
    for i in range(NumStations):

        # Si no existe la estacion en el grafo, se añade
        if stations[i] not in metro_network.keys():
            if i != NumStations - 1:
                metro_network[stations[i]] = [(stations[i+1], travel_times[i+1])]
            else:
                metro_network[stations[i]] = [(stations[i-1], travel_times[i-1])]
        # Sí si existe, añadimos los elementos correspondientes a dada estacion
        else:
            if i != NumStations - 1:
                metro_network[stations[i]].append((stations[i+1], travel_times[i+1]))
            else:
                metro_network[stations[i]].append((stations[i-1], travel_times[i-1]))

# El grafo aún no contempla que, si A conecta con B entonces B conecta con A
# Iniciamos la busqueda sobre todos los nodos
for node, connections in list(metro_network.items()):
    # Exploramos cada nodo objetivo dada las conexiones actuales
    for target, time in connections:

        # Si el objetivo no está en las llaves del grafo, se añade
        if target not in metro_network:
            metro_network[target] = []

        # Caso contrario, se añade un nodo restante
        nodes_list, _ = zip(*metro_network[target])
        if node not in nodes_list:
            metro_network[target].append((node, time))

In [46]:
G = nx.Graph()

for node, edges in metro_network.items():
    for edge in edges:
        target, weight = edge
        G.add_edge(node, target, weight=weight.total_seconds())

node_degrees = dict(G.degree())

max_degree = max(node_degrees.values())
min_degree = min(node_degrees.values())

node_colors = [node_degrees[node] for node in G.nodes()]

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = location_stations[edge[0]]
    x1, y1 = location_stations[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

node_x = []
node_y = []
node_text = []
for node in G.nodes():
    x, y = location_stations[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"{node}\n# de estaciones: {node_degrees[node]}")

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='gray'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    textposition="top center",
    marker=dict(
        showscale=True,
        colorscale='turbo',
        cmin=min_degree,
        cmax=max_degree,
        color=node_colors,
        size=10,
        colorbar=dict(
            thickness=15,
            title='# de estaciones conectadas',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Visualización de la red de metro de la CDMX',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper")],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

fig.show()

¿Posible matriz de distancias?

In [48]:
matrix_distance = dict()

network_nodes = G.nodes()
for origin_node in network_nodes:
    matrix_distance[origin_node] = []
    for target_node in network_nodes:
        distance_by_origin = nx.shortest_path_length(G, origin_node, target_node, weight="weight")
        matrix_distance[origin_node].append(distance_by_origin)
 
matrix_distance = pd.DataFrame(matrix_distance)
matrix_distance.index = network_nodes

matrix_distance

OBSERVATORIO  TACUBAYA  JUANACATLAN  SANPEDROPINOS  PATRIOTISMO  \
0             0.0     170.0        324.0          291.0        300.0   
1           170.0       0.0        154.0          121.0        130.0   
2           324.0     154.0          0.0          275.0        284.0   
3           291.0     121.0        275.0            0.0        251.0   
4           300.0     130.0        284.0          251.0          0.0   
..            ...       ...          ...            ...          ...   
160        2914.0    2744.0       2898.0         2663.0       2614.0   
161        3120.0    2950.0       3104.0         2869.0       2820.0   
162        3313.0    3143.0       3297.0         3062.0       3013.0   
163        3486.0    3316.0       3470.0         3235.0       3186.0   
164        3659.0    3489.0       3643.0         3408.0       3359.0   

     CONSTITUYENTES  CHAPULTEPEC  SEVILLA  INSURGENTES  CUAUHTEMOC  ...  \
0             283.0        456.0    523.0        622.0       733.0  ...   
1             113.0        286.0    353.0        452.0       563.0  ...   
2             267.0        132.0    199.0        298.0       409.0  ...   
3             234.0        407.0    474.0        573.0       684.0  ...   
4             243.0        416.0    483.0        582.0       579.0  ...   
..              ...          ...      ...          ...         ...  ...   
160          2857.0       2844.0   2777.0       2678.0      2567.0  ...   
161          3063.0       3050.0   2983.0       2884.0      2773.0  ...   
162          3256.0       3243.0   3176.0       3077.0      2966.0  ...   
163          3429.0       3416.0   3349.0       3250.0      3139.0  ...   
164          3602.0       3589.0   3522.0       3423.0      3312.0  ...   

     SANANDRESTO  LOMASESTRELLA  CALLE11  PERIFERICOOTE  TEZONCO  OLIVOS  \
0         2117.0         2282.0   2425.0         2596.0   2827.0  2914.0   
1         1947.0         2112.0   2255.0         2426.0   2657.0  2744.0   
2         2101.0         2266.0   2409.0         2580.0   2811.0  2898.0   
3         1866.0         2031.0   2174.0         2345.0   2576.0  2663.0   
4         1817.0         1982.0   2125.0         2296.0   2527.0  2614.0   
..           ...            ...      ...            ...      ...     ...   
160        797.0          632.0    489.0          318.0     87.0     0.0   
161       1003.0          838.0    695.0          524.0    293.0   206.0   
162       1196.0         1031.0    888.0          717.0    486.0   399.0   
163       1369.0         1204.0   1061.0          890.0    659.0   572.0   
164       1542.0         1377.0   1234.0         1063.0    832.0   745.0   

     NOPALERA  ZAPOTITLAN  TLALTENCO  TLAHUAC  
0      3120.0      3313.0     3486.0   3659.0  
1      2950.0      3143.0     3316.0   3489.0  
2      3104.0      3297.0     3470.0   3643.0  
3      2869.0      3062.0     3235.0   3408.0  
4      2820.0      3013.0     3186.0   3359.0  
..        ...         ...        ...      ...  
160     206.0       399.0      572.0    745.0  
161       0.0       193.0      366.0    539.0  
162     193.0         0.0      173.0    346.0  
163     366.0       173.0        0.0    173.0  
164     539.0       346.0      173.0      0.0  

[165 rows x 165 columns]